In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [2]:
qa_df = pd.read_csv('Quality Assessment.csv')
# get the list of eye id and image id tuple
eye_id_list = list(qa_df['Eye ID'])
image_id_list = list(qa_df['Image ID'])
# make tuple
eye_image_id_list = list(zip(eye_id_list, image_id_list))
print(eye_image_id_list[:2])

qa_df

[(1, 0), (1, 1)]


,Eye ID,Image ID,Low signal strength,Signal shield,Image blur
0,1,0,0,0,0
1,1,1,0,0,0
2,1,2,0,0,0
3,1,3,0,0,0
4,1,4,0,0,0
...,...,...,...,...,...
3854,125,14,0,0,0
3855,125,15,0,1,0
3856,125,16,0,1,0
3857,125,17,0,1,0


In [2]:
demo_df = pd.read_csv('Demographics of the participants.csv')
demo_df

,Patient ID,Eye ID,Eye category,Age(years),Sex,Stage
0,1,1,OD,58,Female,4
1,2,2,OS,76,Female,4
2,3,3,OS,70,Female,3
3,4,4,OS,53,Female,4
4,5,5,OS,65,Female,3
...,...,...,...,...,...,...
120,115,121,OD,67,Male,4
121,116,122,OS,59,Female,3
122,117,123,OS,64,Female,3
123,118,124,OD,45,Female,4


In [20]:
name_list = []
num_frames_set = set()
data_dir = 'Images/'
for row, values in demo_df.iterrows():
    pat_id = values['Patient ID']
    eye_id = values['Eye ID']
    stage = values['Stage']
    frame_list = os.listdir(data_dir + f'{eye_id}')
    num_frames = len(frame_list)
    if num_frames == 48:
        print(eye_id, stage)

    num_frames_set.add(num_frames)
    name = f'{eye_id}_stage_{stage}_{pat_id}'
    name_list.append((name, stage, pat_id, eye_id))
print(num_frames_set)
print(name_list[:2])
# save the name list
name_df = pd.DataFrame(name_list, columns=['name', 'stage', 'pat_id', 'eye_id'])
name_df.to_csv('name_list.csv', index=False)

94 3
{32, 97, 34, 37, 73, 48, 49, 17, 19, 115, 25, 63}
[('1_stage_4_1', 4, 1, 1), ('2_stage_4_2', 4, 2, 2)]


get the split information from metadata

In [4]:
patient_id = demo_df['Patient ID'].tolist()
stage = demo_df['Stage'].tolist()
eye_id = demo_df['Eye ID'].tolist()
# get the mapping of eye id and patient id into a dictionary
eye_patient_dict = dict(zip(eye_id, patient_id))
print(eye_patient_dict)


# get train val split using 8:2, using random seed 42
np.random.seed(42)
train_patient_id = np.random.choice(patient_id, int(len(patient_id)*0.8), replace=False)
val_patient_id = [i for i in patient_id if i not in train_patient_id]

# get train val eye id list, get eye id from demo_df
train_eye_id = []
val_eye_id = []

for i in range(len(patient_id)):
    if patient_id[i] in train_patient_id:
        train_eye_id.append(eye_id[i])
    else:
        val_eye_id.append(eye_id[i])

print('train eye id:', train_eye_id)
print('val eye id:', val_eye_id)



{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 54, 57: 55, 58: 56, 59: 57, 60: 58, 61: 59, 62: 60, 63: 61, 64: 62, 65: 63, 66: 64, 67: 65, 68: 66, 69: 67, 70: 68, 71: 69, 72: 70, 73: 71, 74: 72, 75: 73, 76: 74, 77: 75, 78: 76, 79: 76, 80: 77, 81: 78, 82: 79, 83: 80, 84: 81, 85: 82, 86: 83, 87: 84, 88: 85, 89: 86, 90: 87, 91: 88, 92: 89, 93: 90, 94: 91, 95: 92, 96: 93, 97: 94, 98: 95, 99: 96, 100: 97, 101: 98, 102: 99, 103: 100, 104: 100, 105: 101, 106: 102, 107: 103, 108: 104, 109: 105, 110: 106, 111: 107, 112: 108, 113: 109, 114: 109, 115: 110, 116: 111, 117: 112, 118: 112, 119: 113, 120: 114, 121: 115, 122: 116,

In [ ]:
# save train val eye id list to txt 
with open('train_eye_id.txt', 'w') as f:
    for item in train_eye_id:
        f.write("%s\n" % item)

with open('val_eye_id.txt', 'w') as f:
    for item in val_eye_id:
        f.write("%s\n" % item)

# save train val patient id list to txt
with open('train_patient_id.txt', 'w') as f:
    for item in train_patient_id:
        f.write("%s\n" % item)

with open('val_patient_id.txt', 'w') as f:
    for item in val_patient_id:
        f.write("%s\n" % item)


In [5]:
# load train val eye id list from txt
with open('train_eye_id.txt', 'r') as f:
    train_eye_id = f.read().splitlines()

with open('val_eye_id.txt', 'r') as f:
    val_eye_id = f.read().splitlines()

# load train val patient id list from txt
with open('train_patient_id.txt', 'r') as f:
    train_patient_id = f.read().splitlines()

with open('val_patient_id.txt', 'r') as f:
    val_patient_id = f.read().splitlines()
    

In [14]:
# initialize a csv file with fname,split,dataset,image_path,label_path,weight
csv_file = 'index.csv'
columns = ['fname', 'split', 'dataset', 'image_path', 'label_path', 'weight']
df = pd.DataFrame(columns=columns)
dataset = 'oimhs'
weight = 1
OIMHS_dataset = 'OIMHS_dataset'
for eye_id, image_id in eye_image_id_list:
    if str(eye_id) in train_eye_id:
        split = 'train'
    else:
        split = 'val'
    fname = str(image_id) + '.png'
    oct_fname = 'oct-' + fname
    mask_fname = 'mask-' + fname

    image_path = f'{OIMHS_dataset}/Processed_Images/{eye_id}/{oct_fname}'
    label_path = f'{OIMHS_dataset}/Processed_Images/{eye_id}/{mask_fname}'
    # add row to df
    df = df._append({'fname': fname, 'split': split, 'dataset': dataset, 'image_path': image_path, 'label_path': label_path, 'weight': weight}, ignore_index=True)

df.to_csv(csv_file, index=False)

